In [ ]:
!pip install typing-extensions  ipython 

/bin/bash: /home/akv/Documents/practical-ai/Pro-Agent-Hugfce/agenv1/bin/pip: /home/akv/Documents/practical-ai/pro-agent-hugfce/agenv1/bin/python: bad interpreter: No such file or directory


In [7]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from typing import Annotated
from operator import add


# Graph state
class State(TypedDict):
    code: str
    description: str
    testcases: str
    reviewed_code:str
    code_approved:str

ModuleNotFoundError: No module named 'langgraph'


AI agent using LangGraph and Groq API

What Does My Agent Do?

Designed to 
* Generate code
* Write test cases 
* Review code
* Manage approvals

automated code review system. It uses an LLM to generate code, write test cases, suggest improvements, and finally, simulate a managerial approval process. The workflow is visually represented as a DAG, enhancing understanding and maintainability. 
The system is designed to iterate if the manager rejects the code, sending it back for further development and review.

In [ ]:
!pip install dotenv

In [ ]:
!pip install langchain-groq

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [ ]:
llm=ChatGroq(model="qwen-2.5-32b")
#llm = ChatOpenAI(model="gpt-4o")
result=llm.invoke("who are you")
result

In [ ]:
# Nodes
def developer(state: State):
    """First LLM call to generate code"""
    msg = llm.invoke(f"Write a code in programming language {state['description']}")
    return {"code": msg.content}

def check_approved(state: State):
    """Gate function to check if the code approved or not"""
    if "approved" in state["code_approved"] or "pass code" in state["code_approved"]:
        return "Pass"
    return "Fail"

# def code_improvements(state:State):
#     """check if there are any improvements suggestions given"""
#     if "improved" in state["reviewed_code"]:
#         return "Fail"
#     return "Pass"

def write_testcases(state: State):
    """llm call to write the test cases for the code"""
    msg = llm.invoke(f"can you please write test cases for the following code: {state['code']}")
    return {"testcases": msg.content}


def reviewer(state: State):
    """Second LLM call to review  the code"""
    msg = llm.invoke(f"Make this code more efficient by giving suggestions with suggestions: {state['code']}")
    print(msg.content)
    return {"reviewed_code": msg.content}


def manager(state: State):
    """Third LLM call for manager review"""
    msg = llm.invoke(f"act like a manager on the code given by reviewer and approves if it looks code good 'Pass' else gives 'Fail': {state['reviewed_code']}")
    return {"code_approved": msg.content}


# Build workflow
workflow = StateGraph(State)

# Add nodes
workflow.add_node("developer", developer)
workflow.add_node("write_testcases", write_testcases)
workflow.add_node("reviewer", reviewer)
workflow.add_node("manager", manager)

# Add edges to connect nodes
workflow.add_edge(START, "developer")
workflow.add_edge("developer", "write_testcases")
workflow.add_edge("write_testcases", "reviewer")
#give suggestions for code improvements
# workflow.add_conditional_edges(
#     "reviewer",code_improvements, {"Fail": "developer", "Pass": "manager"}
# )
workflow.add_conditional_edges(
    "manager",check_approved, {"Fail": "developer", "Pass": END}
)
workflow.add_edge("reviewer", "manager")
workflow.add_edge("manager", END)

# Compile
chain = workflow.compile()

# Show workflow
display(Image(chain.get_graph().draw_mermaid_png()))

# Invoke
state = chain.invoke({"description": "write a python code  to print first 10 numbers"})
print("\n--- --- ---\n")
if 'code' in state:
    print(state['code'])
else:
    print("code is not present")
    
if "testcases" in state:
    print(state['testcases'])
    print("\n--- --- ---\n")

    print("Final code:")
else:
    print(" gate - test no code detected!")

if "code_approved" in state:
    print("code_approved.....")
    print(state["code_approved"])